In [ ]:
import os
# DECLARE HOW MANY GPUS YOU WISH TO USE. 
# KAGGLE ONLY HAS 1, BUT OFFLINE, YOU CAN USE MORE
os.environ["CUDA_VISIBLE_DEVICES"]="0" #0,1,2,3 for four gpu

# VERSION FOR SAVING/LOADING MODEL WEIGHTS
# THIS SHOULD MATCH THE MODEL IN LOAD_MODEL_FROM
VER=16

# IF VARIABLE IS NONE, THEN NOTEBOOK COMPUTES TOKENS
# OTHERWISE NOTEBOOK LOADS TOKENS FROM PATH
LOAD_TOKENS_FROM = '../input/tf-longformer-v12'

# IF VARIABLE IS NONE, THEN NOTEBOOK TRAINS A NEW MODEL
# OTHERWISE IT LOADS YOUR PREVIOUSLY TRAINED MODEL
LOAD_MODEL_FROM = '../input/tf-ver3'
# LOAD_MODEL_FROM=None 
# IF FOLLOWING IS NONE, THEN NOTEBOOK 
# USES INTERNET AND DOWNLOADS HUGGINGFACE 
# CONFIG, TOKENIZER, AND MODEL
DOWNLOADED_MODEL_PATH = '../input/tf-longformer-v12'

if DOWNLOADED_MODEL_PATH is None:
    DOWNLOADED_MODEL_PATH = 'model'    
MODEL_NAME = 'allenai/longformer-base-4096'

In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# from transformers import *
import transformers
print('TF version',tf.__version__)
print('TF version',transformers.__version__)

In [ ]:
# USE MULTIPLE GPUS

if os.environ["CUDA_VISIBLE_DEVICES"].count(',') == 0:
    strategy = tf.distribute.get_strategy()
    print('single strategy',strategy)
else:
    strategy = tf.distribute.MirroredStrategy()
    print('multiple strategy')

In [ ]:
strategy

In [ ]:
# tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
# !nvidia-smi

In [ ]:
INPUT='/kaggle/input'
import glob
train_df = pd.read_csv(f'{INPUT}/feedback-prize-2021/train.csv')
def get_raw_text(x):
    with open(f'{INPUT}/feedback-prize-2021/train/{x}.txt',"r") as f:
      return f.read()
texts=train_df.groupby('id')['id'].first().apply(get_raw_text).reset_index(name="text")
train_df =pd.merge(train_df, texts, how='inner', on='id')
del texts 

test_df = []
for i in glob.glob(f"{INPUT}/feedback-prize-2021/test/*.txt"):
  with open(i,"r") as f:
    test_df.append(
        {'id':os.path.basename(i)[:-4],'text':f.read()}
    )
test_df = pd.DataFrame(test_df)
# Step 01 split to sentences 

import glob
import re
import numpy as np
def word_to_sentence(value):
  text = value + "."
  sentence = []
  for m in re.finditer(r"((\n\n)|([\.?!]))|(\bBecause)|(\bbecause\b)|(\bthe\b)|(\bThe\b)|(\bIf\b)|(\bif\b)|(\bstudents\b)|(\bStudents\b)|(\bfirst\b)|(\bsecond\b)|(\bFirst\b)|(\bSecond\b)|(\bwe\b)|(\bthey\b)|(\bWe\b)|(\bThey\b)|(\baccording\b)|(\bAccording\b)",text):
    if len(sentence) == 0:
      sentence.append((0,m.start(),m.end()))
    else:
      
      sentence.append((sentence[-1][1],m.start(),m.end()))
  sentence = np.array([(i[0],i[1]) for i in sentence if i[1] - i[0] > 0])
  return sentence

def word_per_sentence(value):
  sentence = value.sentene
  words = []
  for i in sentence:
    words.append(len(value.text[i[0]:i[1]].split()))
  return words

import multiprocessing as mp

with mp.Pool(mp.cpu_count()) as pool:
    sentence = pool.map(word_to_sentence, train_df.groupby('id')['text'].first())
    sentence = [{ 'id':i,'sentence':s} for i,s in zip(train_df.groupby('id')['id'].first(), sentence)]
    sentence = pd.DataFrame(sentence)
    train_df = pd.merge(train_df, sentence, how='inner', on='id')
    del sentence
    pool.close()
ab = test_df.groupby('id').first()
abs = []
for i in range(ab.shape[0]):
  # print(i)
  sentence = word_to_sentence(ab.iloc[i].text)
  abs.append(sentence)

abs = [{ 'id':i,'sentence':s} for i,s in zip(test_df.groupby('id')['id'].first(), abs)]
sentence = pd.DataFrame(abs)
test_df = pd.merge(test_df, sentence, how='inner', on='id')
del sentence
del abs
del ab


In [ ]:
word_to_sentence("oadsa becau we in are")

In [ ]:
train_df.head(2)

In [ ]:
# train_df = train_df.sample(200)

In [ ]:
# step 03 convert to Labels
def text_to_sentences_map(example):
  """return
    +> example:copy value
    //+> position_strings  = list[str] : postion_string[i] = predictionstring[i]
    +> sentences = list[str] :  sentences[i] = sentence[i]
    +> offset = list[tuple(int,int)] : offset[i] = (start, end) : char level sentence[i]
  """
  text = example.text
  sentence = example.sentence 
  # position_string = []
  sentences = []
  offsets = []
  ss = []
  for i in range(len(sentence)):
    t = text[sentence[i][0] : sentence[i][1]]
    re_punc = re.compile("([\"\''().,;:/_?!—\-])")
    t = re_punc.sub(r" ", t)
    if len(t.split()) >= 1:
      # merger to next 
      ss.append(sentence[i])
    else:
      if i+ 1 < len(sentence):
        sentence[i+1][0] = sentence[i][0]
    # if sentence[i][1] - sentence[i][0] < 
  for start,end  in ss:
    offsets.append((start, end))
    sentences.append(text[start:end])
  return {
      'sentences':sentences,
      # 'position_strings':position_string,
      'offset':offsets,
      'id':example.id 
  }

import tqdm
a=[]
ab = train_df.groupby('id').first().reset_index()
for i in tqdm.tqdm(range(ab.shape[0]),total=ab.shape[0]):
  a.append(text_to_sentences_map(ab.iloc[i]))
del ab
ab =pd.DataFrame(a)
print(ab)
train_df = pd.merge(train_df, ab, on='id', how='inner')
train_df.head(1)
df1 = train_df.groupby('id')['discourse_start'].apply(list).reset_index(name="starts")
df2 = train_df.groupby('id')['discourse_end'].apply(list).reset_index(name="ends")
df3 = train_df.groupby('id')['discourse_type'].apply(list).reset_index(name="classes")
df4 = train_df.groupby('id')['predictionstring'].apply(list).reset_index(name="predictionstrings")
train_df = pd.merge(train_df, df1, on='id',how='inner')
train_df = pd.merge(train_df, df2, on='id',how='inner')
train_df = pd.merge(train_df, df3, on='id',how='inner')
train_df = pd.merge(train_df, df4, on='id',how='inner')
train_df = train_df[['id','discourse_id','discourse_start','discourse_text','discourse_end','starts','ends', 'classes','sentence','sentences','offset','text','predictionstring','predictionstrings','discourse_type']]

In [ ]:
a=[]
ab = test_df.groupby('id').first().reset_index()
for i in tqdm.tqdm(range(ab.shape[0]),total=ab.shape[0]):
  a.append(text_to_sentences_map(ab.iloc[i]))
ab =pd.DataFrame(a)
test_df = pd.merge(test_df, ab, on='id', how='inner')
del ab
del df1
del df2
del df3
del a


In [ ]:
train_df.iloc[20].sentences

In [ ]:
train_df.iloc[20].text

In [ ]:
# ids = train_df['id'].unique()[:100]
# index = train_df['id'].apply(lambda x:x in ids)
# train_df = train_df[index]

In [ ]:
# train = train_df.copy()
# train_df = train_df.groupby('id').first().reset_index()

In [ ]:
# train

In [ ]:
from collections import Counter
def bb_intersection_over_union2(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	# yA = max(boxA[1], boxB[1])
	xB = min(boxA[1], boxB[1])
	# yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) 
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[1] - boxA[0] + 1) 
	boxBArea = (boxB[1] - boxB[0] + 1) 
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea )
	# return the intersection over union value
	return iou
def finding(x):
  sentence = x.sentence
  discourse_start = x.discourse_start
  discourse_end = x.discourse_end
  iou=0
  for x in sentence:
    iou = max(iou,bb_intersection_over_union2(x, (discourse_start,discourse_end )) )
    # if iou >=0.9:
      # break
  return iou
# a=train_df.apply(finding, axis=1)

# CODE FROM : Rob Mulla @robikscube
# https://www.kaggle.com/robikscube/student-writing-competition-twitch
def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    set_pred = set(row.predictionstring_pred.split(' '))
    set_gt = set(row.predictionstring_gt.split(' '))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter/ len_pred
    return [overlap_1, overlap_2]


def score_feedback_comp(pred_df, gt_df):
    """
    A function that scores for the kaggle
        Student Writing Competition
        
    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = gt_df[['id','discourse_type','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df = pred_df[['id','class','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df.index
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(gt_df,
                           left_on=['id','class'],
                           right_on=['id','discourse_type'],
                           how='outer',
                           suffixes=('_pred','_gt')
                          )
    joined['predictionstring_gt'] = joined['predictionstring_gt'].fillna(' ')
    joined['predictionstring_pred'] = joined['predictionstring_pred'].fillna(' ')

    joined['overlaps'] = joined.apply(calc_overlap, axis=1)

    # 2. If the overlap between the ground truth and prediction is >= 0.5, 
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    joined['overlap1'] = joined['overlaps'].apply(lambda x: eval(str(x))[0])
    joined['overlap2'] = joined['overlaps'].apply(lambda x: eval(str(x))[1])


    joined['potential_TP'] = (joined['overlap1'] >= 0.5) & (joined['overlap2'] >= 0.5)
    joined['max_overlap'] = joined[['overlap1','overlap2']].max(axis=1)
    tp_pred_ids = joined.query('potential_TP') \
        .sort_values('max_overlap', ascending=False) \
        .groupby(['id','predictionstring_gt']).first()['pred_id'].values

    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = [p for p in joined['pred_id'].unique() if p not in tp_pred_ids]

    matched_gt_ids = joined.query('potential_TP')['gt_id'].unique()
    unmatched_gt_ids = [c for c in joined['gt_id'].unique() if c not in matched_gt_ids]

    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    #calc microf1
    my_f1_score = TP / (TP + 0.5*(FP+FN))
    return my_f1_score

def calc_word_indices(full_text, discourse_start, discourse_end):
    start_index = len(full_text[:discourse_start].split())
    token_len = len(full_text[discourse_start:discourse_end].split())
    output = list(range(start_index, start_index + token_len))
    if output[-1] >= len(full_text.split()):
        output = list(range(start_index, start_index + token_len-1))
    return " ".join(str(x) for x in output)

def refine(pred, offset):
  target_map_rev = {0:'Lead', 2:'Position', 4:'Evidence', 6:'Claim', 8:'Concluding Statement',
             10:'Counterclaim', 12:'Rebuttal', 14:'blank'}

  pred = pred 
  # print(pred)
  res = []
  i=0
  M = min(len(offset), len(pred))
  while i < M:
    start = pred[i]
    prediction = []
    if start in [0,2,4,6,8,10,12]:
        prediction.append(i)
        i += 1
        if i>=M: break
        while i < M and pred[i]==start+1:
            prediction.append(i)
            i += 1
            if i>=M: break
        # print(prediction)
        res.append({
            'class':target_map_rev[int(pred[prediction[0]]) ],
            'offset':(offset[prediction[0]][0], offset[prediction[-1]][1])
        })
    else:
        if start != 14:
          start  = start  - 1
          pred[i] -= 1
          prediction.append(i)
          i += 1
          if i>=M: break
          while i < M and pred[i]==start+1:
              prediction.append(i)
              i += 1
              if i>=M: break
          # print(prediction)
          res.append({
              'class':target_map_rev[int(pred[prediction[0]]) ],
              'offset':(offset[prediction[0]][0], offset[prediction[-1]][1])
          })
        else:
          i += 1
  return res

# def end_to_end_file(txt):


# class IOUCallBack(tf.keras.callbacks.Callback):
#   def __init__(self, model, valid_df):
#     self.model = model
#     self.val_df = valid_df.copy()
#     self.instancedf = valid_df.groupby("id").first().reset_index()
#     train_sentences = self.instancedf['sentences'].values
#     train_labels = self.instancedf['labels'].values

#     val_set = tf.data.Dataset.from_generator(partial(gernerator,sentences=train_sentences, labels=train_labels ), 
#                                            output_signature=(
#                                                {
#                                                               'tokens':tf.TensorSpec(shape=(1024), dtype=tf.int32),
#                                                               'attention':tf.TensorSpec(shape=(1024), dtype=tf.int32),
#                                                               'sentence':tf.TensorSpec(shape=(1024,MAX_SENTENCE), dtype=tf.float32)
#                                               },
#                                                              tf.TensorSpec(shape=(MAX_SENTENCE), dtype=tf.int32))
                                           
#                                            )
#     self.val_set=val_set.batch(
#       2, drop_remainder=False, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False,
#       name=None
#   ).prefetch(4)
#   self.predictionstrings=None
#   def on_epoch_end(self, batch, logs=None):
#     print("\n\nevaluated validation set")
#     predict = self.model.predict(self.val_set)
#     predict = np.argmax(predict,-1)
#     print("predict done : ", predict.shape)
#     print(predict)
#     predictionstrings =[]
#     for i in range(len(self.instancedf)):
#       sample =self.instancedf.iloc[i]
#       offset = sample.offset 
#       text = sample.text
#       res = refine(predict[i], offset)
#       for r in res:
#         predictionstrings.append(
#             {
#                 "predictionstring":calc_word_indices(text, r['offset'][0],r['offset'][1]),
#                 "id":sample.id,
#                 "class":r['class']
#             }

#         )
#     oof=pd.DataFrame(predictionstrings)
#     f1s = []
#     if oof.shape[0] ==0:
#       print("predict none")
#       return
#     CLASSES = oof['class'].unique()
#     for c in CLASSES:
#       pred_df = oof.loc[oof['class']==c].copy()
#       gt_df = self.val_df.loc[self.val_df['discourse_type']==c].copy()
#       f1 = score_feedback_comp(pred_df, gt_df)
#       print(c,f1)
#       f1s.append(f1)
#     self.predictionstrings = oof
#     print()
#     print('Overall',np.mean(f1s))
#     print("to prediction strings\n\n")

from collections import Counter
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	# yA = max(boxA[1], boxB[1])
	xB = min(boxA[1], boxB[1])
	# yB = min(boxA[3], boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) 
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[1] - boxA[0] + 1) 
	boxBArea = (boxB[1] - boxB[0] + 1) 
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea )
	# return the intersection over union value
	return iou

# a = [1936, 2401, 2916, 4761, 9216, 9216, 9604, 9801] 

# c = Counter(a)

# print(c.most_common(1)) # the one most common element... 2 would mean the 2 most common
# [(9216, 2)] # a set containing the element, and it's count in 'a'

def provide_label_sentence(example):
  sentence = example.sentences
  labels = []
  z=[0 for i in example.classes]

  for offset in example.offset:
    st = offset[0]
    end = offset[1]
    b=[]
    for i in range(len(example.starts)):
      sts = example.starts[i]
      ends = example.ends[i]
      # if (abs(sts - st) + abs(end - ends) < 6 + 6) or (abs(sts - st) < 6 and end <= ends and end > sts) or (sts < st and
      #                                                                                                    st < ends and abs(end-ends) < 6) or (sts<=st and end<=ends):
      #   b.append(i)
      iou = bb_intersection_over_union(offset, (sts,ends))
      iou2 = bb_intersection_over_union((sts,ends),offset)
      if  iou>= 0.5 or iou2>=0.5 :
          b.append((i,max(iou,iou2)))
    if len(b) > 1:
      b = sorted(b, key=lambda x:x[1])
#       print(example.id)
      # cc = Counter(b)
      # labels.append(cc.most_common(1)[0][0])
      labels.append(b[-1][0])
    elif len(b):
      labels.append(b[0][0])
    if len(b) == 0:
      labels.append(-1)
  refine = []
  for i in range(len(labels)):
    if labels[i] == -1:
      refine.append("O")
    else:
      l = example.classes[labels[i]]
      if i==0 or labels[i-1] != labels[i]:
        refine.append('B-' + l)
      else:
        refine.append("I-" +l)
        # print(example)
  return refine

In [ ]:
# provide_label_sentence(train_df.iloc[0])

In [ ]:
# train_df.iloc[0]

In [ ]:
# train[train['id']=='09FF4F0D2B79']
!pip install multiprocesspandas

In [ ]:
# labels= []
# for i in tqdm.tqdm(range(train_df.shape[0])):
#   # print(i)
#   labels.append(provide_label_sentence(train_df.iloc[i]))
train = train_df.copy()
train_df = train_df.groupby('id').first().reset_index()
import time
from multiprocesspandas import applyparallel
# train_texts, valid, oof = None, None, None
st=time.time()
labels= train_df.apply_parallel(provide_label_sentence, num_processes=30, axis=0)
# for i in tqdm.tqdm(range(train_df.shape[0])):
#   # print(i)
#   labels.append(provide_label_sentence(train_df.iloc[i]))
print(time.time() - st)

In [ ]:
train_df.shape

In [ ]:
train_df['labels'] = labels
del labels

In [ ]:
# print(train_df[train_df['id'] == 'ECA4D673EC46'].iloc[0])
train_df.head(1)

In [ ]:
# print(train_df[train_df['id'] == 'ECA4D673EC46'].iloc[0].text[162:276])

In [ ]:
# print(train_df[train_df['id'] == 'ECA4D673EC46'].iloc[0].text[276:339])

In [ ]:
# print(train_df[train_df['id'] == 'ECA4D673EC46'].iloc[0].offset)

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
print(MODEL_NAME)

In [ ]:
# co cac labels tokenizer text 
# step 01 :  processing sentence 
# step 02 :  join sentence 
# step 03:   get char level index sentence 
# step 04:   tokenizer transformer 
# step 05:   count total token persentence with char level index 
# Affter step 05: input_ids = [1,1024], sentences_ids = [1024, 160] = [1,160] 

In [ ]:
MAX_LENGTH  = 1024
MAX_SENTENCE = 400
DICT_LABEL= {'B-Lead':0,'I-Lead':1, 'B-Position':2,'I-Position':3, 'B-Evidence':4, 'I-Evidence':5, 'B-Claim':6,  'I-Claim':7,
             'B-Concluding Statement':8,'I-Concluding Statement':9,
             'B-Counterclaim':10, 'I-Counterclaim':11,'B-Rebuttal':12,'I-Rebuttal':13,'O':14,"pad":-1}
def clean(sent):
  re_punc = re.compile("([\"\''().,;:/_?!—\-])")
  sent = re_punc.sub(r" ", sent)
  # todos: +> augment online - augmen offline: word level 
  return sent 
def share_clean(sentences):
  sentences = [clean(s) + "." for s in sentences][:MAX_SENTENCE]
  # todos : augment sentence level 
  # todos: augment NER sentence level 
  # todos: make it offline possible to handle outline
  char_level = []
  for s in sentences:
    if len(char_level)==0: 
      char_level.append(len(s))
    else:
      char_level.append(char_level[-1]  + len(s))
  sentence = "".join(sentences)

  tokens = tokenizer.encode_plus(sentence, max_length=MAX_LENGTH, padding='max_length',
                                   truncation=True, return_offsets_mapping=True)
  sentence_mask  =  [ [0.,] * 1024 for i in  range(MAX_SENTENCE)]
  index_sent = 0
  for index_w,o in enumerate(tokens['offset_mapping']):
    if o == (0,0) and index_w: # padding and <s> token
      continue 
    while o[1] > char_level[index_sent] or o[0] > char_level[index_sent]:
      index_sent += 1
    if o[1] <= char_level[index_sent]:
      sentence_mask[index_sent][index_w]  = 1
    else:
      print(index_sent, index_w,o[1])
  # print(index_sent)
  sentence_mask = np.array(sentence_mask)
  sentence_mask  = sentence_mask / (1e-8+np.sum(sentence_mask,-1)[:,None])
  return tokens,sentence_mask.T


In [ ]:
# share_clean(train_df.iloc[2].sentences)
# train_df['x'] = train_df['sentences'].apply(lambda x:len(x))
# train_df[train_df['x'] == 1]

In [ ]:
# print(train_df[train_df['x'] == 1].iloc[0].text)

In [ ]:
# token, mask=share_clean(train_df.iloc[398].sentences)
# mask.shape

In [ ]:
# print(train_df.iloc[398])
import numpy as np

In [ ]:
# IDS=[1,]*16000
# MAX_LENGTH=1024
# MAX_SENTENCE=150
# train_tokens = np.zeros((len(IDS),MAX_LENGTH), dtype='int32')
# train_attention = np.zeros((len(IDS),MAX_LENGTH), dtype='int32')
# train_mask = np.zeros((len(IDS),MAX_LENGTH, MAX_SENTENCE))

In [ ]:
#  tokens = []
#  attentions=[]
#  sentences = []
#  labels = []
# from tensorflow.keras.utils.np_utils import to_categorical   


from functools import partial
def gernerator(sentences, labels, shuffle=False):
  if shuffle:
    idx = np.arange(len(sentences))
    np.random.shuffle(idx)
  else:
    idx = np.arange(len(sentences))
  for index in idx:
    sen = sentences[index]
    l = labels[index]
    token, train_mask=share_clean(sen)
    input_ids=token['input_ids']
    attention_mask = token['attention_mask']
    label= l[:MAX_SENTENCE]
    if len(label) < MAX_SENTENCE:
      label.extend(["pad",]*(MAX_SENTENCE - len(label)))
    labelsx = [DICT_LABEL[i] for i in label] # 1024,1
    labelsx = np.array(labelsx).reshape(-1,)
    # categorical_labels = to_categorical(labels, num_classes=15)
    yield {
        'tokens':input_ids,
        'attention':attention_mask,
        'sentence':train_mask
    }, labelsx


train_sentences = train_df['sentences'].values
train_labels = train_df['labels'].values


IDS = train_df.id.unique()
# print('There are',len(IDS),'train texts.')
# train_tokens = np.zeros((len(IDS),MAX_LENGTH), dtype='int32')
# train_attention = np.zeros((len(IDS),MAX_LENGTH), dtype='int32')
# # train_mask = np.zeros((len(IDS),MAX_LENGTH, MAX_SENTENCE))
# os.makedi
# labels =[]
# for i in tqdm.tqdm(range(train_df.shape[0])):
#     # print(i)
#     token, mask=share_clean(train_df.iloc[i].sentences)
#     # tokens.append(token['input_ids'])
#     # attentions.append(token['attention_mask'])
#     # sentences.append(mask)
#     train_tokens[i,:] = token['input_ids']
#     train_attention[i,:] = token['attention_mask']
#     train_mask[i,:,:] = mask
#     label= train_df.iloc[i].labels[:MAX_SENTENCE]
#     if len(label) < MAX_SENTENCE:
#       label.extend(["O",]*(MAX_SENTENCE - len(label)))
#     labels.extend([DICT_LABEL[i] for i in label])
# labels  = np.array(labels).reshape(train_df.shape[0],-1)

In [ ]:
np.random.seed(42)
train_idx = np.random.choice(np.arange(len(IDS)),int(0.9*len(IDS)),replace=False)
valid_idx = np.setdiff1d(np.arange(len(IDS)),train_idx)
# valid_idx = train_idx
train_df2 = pd.read_csv(f'{INPUT}/feedback-prize-2021/train.csv')
np.random.seed(None)
print('Train size',len(train_idx),', Valid size',len(valid_idx))

In [ ]:
# train_labels

In [ ]:
valid_pd = train_df2.loc[train_df2['id'].isin(IDS[valid_idx])]

In [ ]:
valid_pd = pd.merge(valid_pd,train_df.iloc[valid_idx], on='id',how='left')

In [ ]:
# valid_pd['offset'] = valid_pd['offset_x']
# valid_pd['text'] = valid_pd['text_x']
valid_pd['predictionstring'] = valid_pd['predictionstring_x']
# valid_pd['sentences'] = valid_pd['sentences_y']
valid_pd['labels'] = valid_pd['labels']
valid_pd['discourse_type'] = valid_pd['discourse_type_x']

In [ ]:
valid_pd

In [ ]:
train_set = tf.data.Dataset.from_generator(partial(gernerator,sentences=train_sentences[train_idx], labels=train_labels[train_idx],shuffle=True ), 
                                           output_signature=(
                                               {
                                                              'tokens':tf.TensorSpec(shape=(1024), dtype=tf.int32),
                                                              'attention':tf.TensorSpec(shape=(1024), dtype=tf.int32),
                                                              'sentence':tf.TensorSpec(shape=(1024,MAX_SENTENCE), dtype=tf.float32)
                                              },
                                                             tf.TensorSpec(shape=(MAX_SENTENCE), dtype=tf.int32))
                                           
                                           )
# val_set = tf.data.Dataset.from_generator(partial(gernerator,sentences=train_sentences[valid_idx], labels=train_labels[valid_idx] ), 
#                                            output_signature=(
#                                                {
#                                                               'tokens':tf.TensorSpec(shape=(1024), dtype=tf.int32),
#                                                               'attention':tf.TensorSpec(shape=(1024), dtype=tf.int32),
#                                                               'sentence':tf.TensorSpec(shape=(1024,150), dtype=tf.float32)
#                                               },
#                                                              tf.TensorSpec(shape=(150), dtype=tf.int32))
                                           
#                                            )



In [ ]:
for i in train_set.take(10):
  print(i[1])

In [ ]:
train_set=train_set.batch(
    4, drop_remainder=False, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False,
).prefetch(8)
# val_set=val_set.prefetch(4,).batch(
#     2, drop_remainder=False, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False,
#     name=None
# )

In [ ]:
# train_dfxx

In [ ]:
# CODE FROM : Rob Mulla @robikscube
# https://www.kaggle.com/robikscube/student-writing-competition-twitch
def calc_overlap(row):
    """
    Calculates the overlap between prediction and
    ground truth and overlap percentages used for determining
    true positives.
    """
    set_pred = set(row.predictionstring_pred.split(' '))
    set_gt = set(row.predictionstring_gt.split(' '))
    # Length of each and intersection
    len_gt = len(set_gt)
    len_pred = len(set_pred)
    inter = len(set_gt.intersection(set_pred))
    overlap_1 = inter / len_gt
    overlap_2 = inter/ len_pred
    return [overlap_1, overlap_2]


def score_feedback_comp(pred_df, gt_df):
    """
    A function that scores for the kaggle
        Student Writing Competition
        
    Uses the steps in the evaluation page here:
        https://www.kaggle.com/c/feedback-prize-2021/overview/evaluation
    """
    gt_df = gt_df[['id','discourse_type','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df = pred_df[['id','class','predictionstring']] \
        .reset_index(drop=True).copy()
    pred_df['pred_id'] = pred_df.index
    gt_df['gt_id'] = gt_df.index
    # Step 1. all ground truths and predictions for a given class are compared.
    joined = pred_df.merge(gt_df,
                           left_on=['id','class'],
                           right_on=['id','discourse_type'],
                           how='outer',
                           suffixes=('_pred','_gt')
                          )
    joined['predictionstring_gt'] = joined['predictionstring_gt'].fillna(' ')
    joined['predictionstring_pred'] = joined['predictionstring_pred'].fillna(' ')

    joined['overlaps'] = joined.apply(calc_overlap, axis=1)

    # 2. If the overlap between the ground truth and prediction is >= 0.5, 
    # and the overlap between the prediction and the ground truth >= 0.5,
    # the prediction is a match and considered a true positive.
    # If multiple matches exist, the match with the highest pair of overlaps is taken.
    joined['overlap1'] = joined['overlaps'].apply(lambda x: eval(str(x))[0])
    joined['overlap2'] = joined['overlaps'].apply(lambda x: eval(str(x))[1])


    joined['potential_TP'] = (joined['overlap1'] >= 0.5) & (joined['overlap2'] >= 0.5)
    joined['max_overlap'] = joined[['overlap1','overlap2']].max(axis=1)
    tp_pred_ids = joined.query('potential_TP') \
        .sort_values('max_overlap', ascending=False) \
        .groupby(['id','predictionstring_gt']).first()['pred_id'].values

    # 3. Any unmatched ground truths are false negatives
    # and any unmatched predictions are false positives.
    fp_pred_ids = [p for p in joined['pred_id'].unique() if p not in tp_pred_ids]

    matched_gt_ids = joined.query('potential_TP')['gt_id'].unique()
    unmatched_gt_ids = [c for c in joined['gt_id'].unique() if c not in matched_gt_ids]

    # Get numbers of each type
    TP = len(tp_pred_ids)
    FP = len(fp_pred_ids)
    FN = len(unmatched_gt_ids)
    #calc microf1
    my_f1_score = TP / (TP + 0.5*(FP+FN))
    return my_f1_score

def calc_word_indices(full_text, discourse_start, discourse_end):
    start_index = len(full_text[:discourse_start].split())
    token_len = len(full_text[discourse_start:discourse_end].split())
    output = list(range(start_index, start_index + token_len))
    if output[-1] >= len(full_text.split()):
        output = list(range(start_index, start_index + token_len-1))
    return " ".join(str(x) for x in output)

def refine(pred, offset):
  target_map_rev = {0:'Lead', 2:'Position', 4:'Evidence', 6:'Claim', 8:'Concluding Statement',
             10:'Counterclaim', 12:'Rebuttal', 14:'blank'}

  pred = pred 
  # print(pred)
  res = []
  i=0
  while i < len(offset):
    start = pred[i]
    prediction = []
    if start in [0,2,4,6,8,10,12]:
        prediction.append(i)
        i += 1
        if i>=len(offset): break
        while i < len(offset) and pred[i]==start+1:
            prediction.append(i)
            i += 1
            if i>=len(offset): break
        # print(prediction)
        res.append({
            'class':target_map_rev[int(pred[prediction[0]]) ],
            'offset':(offset[prediction[0]][0], offset[prediction[-1]][1])
        })
    else:
        if start != 14:
          start  = start  - 1
          pred[i] -= 1
          prediction.append(i)
          i += 1
          if i>=len(offset): break
          while i < len(offset) and pred[i]==start+1:
              prediction.append(i)
              i += 1
              if i>=len(offset): break
          # print(prediction)
          res.append({
              'class':target_map_rev[int(pred[prediction[0]]) ],
              'offset':(offset[prediction[0]][0], offset[prediction[-1]][1])
          })
        else:
          i += 1
  return res


class IOUCallBack(tf.keras.callbacks.Callback):
  def __init__(self, model, valid_df):
    self.model = model
    self.val_df = valid_df.copy()
    self.instancedf = valid_df.groupby("id").first().reset_index()
    train_sentences = self.instancedf['sentences'].values
    train_labels = self.instancedf['labels'].values

    val_set = tf.data.Dataset.from_generator(partial(gernerator,sentences=train_sentences, labels=train_labels ), 
                                           output_signature=(
                                               {
                                                              'tokens':tf.TensorSpec(shape=(1024), dtype=tf.int32),
                                                              'attention':tf.TensorSpec(shape=(1024), dtype=tf.int32),
                                                              'sentence':tf.TensorSpec(shape=(1024,MAX_SENTENCE), dtype=tf.float32)
                                              },
                                                             tf.TensorSpec(shape=(MAX_SENTENCE), dtype=tf.int32))
                                           
                                           )
    self.val_set=val_set.batch(
      2, drop_remainder=False, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False,

  ).prefetch(4)
  def on_epoch_end(self, batch, logs=None):
    print("\n\nevaluated validation set")
    predict = self.model.predict(self.val_set)
    predict = np.argmax(predict,-1)
    print("predict done : ", predict.shape)
    print(predict)
    predictionstrings =[]
    for i in range(len(self.instancedf)):
      sample =self.instancedf.iloc[i]
      offset = sample.offset 
      text = sample.text
      res = refine(predict[i], offset)
      for r in res:
        predictionstrings.append(
            {
                "predictionstring":calc_word_indices(text, r['offset'][0],r['offset'][1]),
                "id":sample.id,
                "class":r['class']
            }

        )
    oof=pd.DataFrame(predictionstrings)
    f1s = []
    if oof.shape[0] ==0:
      print("predict none")
      return
    CLASSES = oof['class'].unique()
    for c in CLASSES:
      pred_df = oof.loc[oof['class']==c].copy()
      gt_df = self.val_df.loc[self.val_df['discourse_type']==c].copy()
      f1 = score_feedback_comp(pred_df, gt_df)
      print(c,f1)
      f1s.append(f1)
    print()
    print('Overall',np.sum(f1s) / 7.)
    print("to prediction strings\n\n")




In [ ]:
from transformers import AutoConfig, TFAutoModel

In [ ]:
# MAX_LENGTH

In [ ]:
def acc(y_true, y_pred):
    mask = tf.where(y_true >= 0, 1, 0)
   
   
    mask = tf.reshape(mask,(-1,1)) 
    mask = tf.cast(mask, tf.float32)
    total = tf.math.reduce_sum(mask) + 0.00001
    y_true  = tf.reshape(y_true,(-1,))
    y_pred = tf.reshape(y_pred,(-1, 15))
    y_pred = tf.math.argmax(y_pred, axis=-1)
    y_pred = tf.reshape(y_pred,(-1,))
    y_true = tf.cast(y_true,tf.int64)
    l = tf.where(y_pred == y_true, 1., 0.)
    l = tf.math.reduce_sum(l) / total
    return l

    # return tf.math.reduce_sum(m) / total

def custom_loss_function(y_true, y_pred):
   mask = tf.where(y_true >= 0, 1, 0)
   
   
   mask = tf.reshape(mask,(-1,1))
   
   mask = tf.cast(mask, tf.float32)
   total = tf.math.reduce_sum(mask) + 0.0001
   y_true  = tf.reshape(y_true,(-1,1))
  #  mask = tf.where(y_true == 14, 0.5, mask)
   y_true  = tf.where(y_true>=0 , y_true, 14)
   y_pred = tf.reshape(y_pred,(-1, 15))
  #  y_pred = tf.math.argmax(y_pred, -1)

   scce = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.SUM)
   lo = scce(y_true, y_pred,sample_weight=mask)
   return tf.math.reduce_sum(lo)/total

 
  #  squared_difference = tf.square(y_true - y_pred)
  #  return tf.reduce_mean(squared_difference, axis=-1)
def mat_mul(a):
  return tf.linalg.matmul(a[1],a[0], transpose_a=True)
def build_model():
    
    tokens = tf.keras.layers.Input(shape=(MAX_LENGTH,), name = 'tokens', dtype=tf.int32)
    attention = tf.keras.layers.Input(shape=(MAX_LENGTH,), name = 'attention', dtype=tf.int32)
    
    mask = tf.keras.layers.Input(shape=(MAX_LENGTH, MAX_SENTENCE), name='sentence', dtype=tf.float32)

    config = AutoConfig.from_pretrained(MODEL_NAME) 
    backbone = TFAutoModel.from_pretrained(MODEL_NAME, config=config)
    x = backbone(tokens, attention_mask=attention)
    x = tf.keras.layers.Lambda(mat_mul)([x[0],mask])
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(15, activation='softmax', dtype='float32')(x)
    
    

    model = tf.keras.Model(inputs=[tokens,attention,mask], outputs=x)
    model.compile(optimizer = tf.keras.optimizers.Adam(lr = 1e-4),
                  loss = [custom_loss_function],
                  metrics = [acc])
    
    return model

In [ ]:

with strategy.scope():
    model = build_model()

In [ ]:
model.load_weights("../input/sentence-level/best_05.hdf5")
model.summary()

In [ ]:
# train_df.iloc[valid_idx]

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:

# valid_pd

In [ ]:
#bs,1024,768
# bs,1024,150

# RNING RATE SCHEDULE AND MODEL CHECKPOINT
EPOCHS = 5
BATCH_SIZE = 4 
LRS = [1e-5, 0.5*1e-5, 0.25*1e-5, 0.25*1e-5, 0.25e-5, 0.25e-5, 0.25e-5, 0.1e-5] 
def lrfn(epoch):
    return LRS[epoch]
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
iou_cb  = IOUCallBack(model, valid_pd)
checkpoint_filepath = './best_{epoch:02d}.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=False)

In [ ]:

# iou_cb.on_epoch_end(1)

In [ ]:
model.fit(train_set,
              # validation_data = val_set,
              callbacks = [lr_callback, model_checkpoint_callback,iou_cb],
              epochs = 4
          )

In [ ]:
# model.fit(train_set,
#           validation_data = val_set,
#           callbacks = [lr_callback, model_checkpoint_callback,iou_cb],
#           epochs = 1
#           )